In [0]:
%run
"/Users/qi.wei1@providence.org/working_folder/concept_proof_codes/add_feature_functions_test/count_meds_pts_number_active_or_alive"

Dictionary for conditions loaded


Dictionary for devices loaded


Dictionary for lab results loaded


Out[239]: 301542

Dictionary for medications loaded


Dictionary for rare diseases loaded


Dictionary for vitals loaded


Dictionary for antibacterials antimycotics loaded


Please always confirm the version of the OMOP table you are using, check the data panel to see if there is a newer version!!!


In [0]:
## Paxlovid
paxlovid_codes = [
  #'2587892', # nirmatrelvir
  #'85762', # ritonavir
  '284640',       # lopinavir / ritonavir
  '2587899', # Paxlovid 5-Day (EUA)
  '2599543'  # paxlovid 150 mg /100 mg dose pack (eua)
]

In [0]:
# ############################################################
# ## All needed parameters
# ############################################################
# ## boolean values to control if only include alive patients, if the diagnosis is first noted within X years, and control for active patient definition
# only_instance_1k, include_only_alive, enable_active_def = True, False, False

# ## start & end date, used for checking whether a patient is active or first noted within a time frame
# ## change the start_date to control past 3 years, 5 years, or 10 years
# start_date, end_date = "2021-12-26", "2022-08-30"

# ###########################################################################
# ## Previous omop_table_version: 2020-08-05, 2021-06-27, 2022-02-11
# ## current omop_table_version: 2022_11_01
# omop_table_version = "2022_11_07"

# med_list, med_name = ['paxlovid'], 'paxlovid'
# ### Run it using function
# paxlovid_df = medication_searching_with_drugNames_possibleRoutes_medName(med_pts_df, med_list, med_name, include_only_alive, enable_active_def, start_date, end_date, only_instance_1k, omop_table_version)

In [0]:
# paxlovid_pts_list = list(paxlovid_df.select('pat_id').toPandas()['pat_id'])
# print(len(paxlovid_pts_list))

In [0]:
from pyspark.sql.functions import lit

start_date, end_date, file_date = "2021-12-26", "2022-08-30", "20211226_20220830_Lancet_bmi"
imids_cohort = spark.sql("""SELECT * FROM rdp_phi_sandbox.qw_IMID_COVID_trainset_cond_med_vax_{}""".format(file_date))
imids_cohort = imids_cohort.withColumn("instance", lit(1000))
# imids_training_df.limit(5).toPandas()

In [0]:
%run
"/Users/qi.wei1@providence.org/working_folder/concept_proof_codes/add_feature_functions_test/add_meds_features_prior_and_after"

Dictionary for conditions loaded


Dictionary for devices loaded


Dictionary for lab results loaded


Out[601]: 301542

Dictionary for medications loaded


Dictionary for rare diseases loaded


Dictionary for vitals loaded


Dictionary for antibacterials antimycotics loaded


Please always confirm the version of the OMOP table you are using, check the data panel to see if there is a newer version!!!


In [0]:
## define whether to filter only instance = 1000
only_instance_1k = True

##omop_table_version
omop_table_version = "2022_06_27"

In [0]:
## Read the table
imids_cohort = spark.sql("SELECT * FROM rdp_phi_sandbox.qw_IMID_COVID_trainset_cond_med_{}".format(file_date))

## Convert the age column from double type into float format
imids_cohort = imids_cohort.withColumn('age', imids_cohort['age'].cast( FloatType() ) )

In [0]:
## 0. load the meds_pts_df from the temp view
tmp =  spark.sql("""select pat_id, instance, rxnormcode, medication_name, orderclass, ordermode, route, rxnormcode, medord_startdate, medord_endate from qw_med_join_table""")
tmp = tmp.withColumnRenamed('pat_id', 'pat_id2').withColumnRenamed('instance', 'instance2')
    
### Directly join with the previous condition dataframe
cond = [imids_cohort.pat_id == tmp.pat_id2, imids_cohort.instance == tmp.instance2]
med_pts_df = imids_cohort.select("pat_id", "instance", "decided_index_date").dropDuplicates().join(tmp, 
                       cond, how = "inner").drop(tmp.pat_id2).drop(tmp.instance2)

In [0]:
possible_routes = ['Oral','Intramuscular', 'Intravenous', 'Subcutaneous Infusion', 'Subcutaneous', 'Intravenous (Continuous Infusion)', 'Rectal']
number_days_after = 14

med_list, med_name = ['paxlovid'], 'paxlovid'
imids_cohort = add_med_after_usage_with_drugNames_possibleRoutes_medName(imids_cohort, med_pts_df, med_list, possible_routes, med_name, number_days_after, only_instance_1k, omop_table_version)

No special treatment to remove codes from paxlovid now, please contact Jenn or the doctor you worked with to confirm.
Filter to include only instance = 1000 records.
paxlovid finished, used approx. 1 minutes


In [0]:
pts_paxlovid = imids_cohort.filter(imids_cohort.after_14_days_paxlovid_logic == 1)
pts_paxlovid.select("pat_id").distinct().count()

Out[723]: 146

In [0]:
pts_paxlovid_df = pts_paxlovid.select("pat_id", "after_14_days_paxlovid_logic").dropDuplicates()

spark.sql("""DROP TABLE IF EXISTS rdp_phi_sandbox.qw_IMID_COVID_trainset_Omicron_Paxlovid_{}""".format(file_date))
table_name = "rdp_phi_sandbox.qw_IMID_COVID_trainset_Omicron_Paxlovid_{}".format(file_date)
pts_paxlovid_df.write.mode("overwrite").format("delta").saveAsTable(table_name)